<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/cat_dog_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.15.0
import os
import tensorflow as tf
print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [2]:
def load_dataset(paths, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    dataset = dataset.map(lambda x, y: (load_and_preprocess_image(x), tf.one_hot(y, depth=2)))
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# 이미지를 숫자화
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [128, 128])
    image = tf.cast(image, tf.float32) / 255.0 #normalization
    return image

In [5]:
path = "/content/drive/MyDrive/세종교육/CNN/data"
files = os.listdir(path)
path_files = [path+f"/{file}" for file in files]
pathfiles = [path+f"/{file}" for file in files]

labels = [0 for _ in range(len(path_files))]
print(labels)


dataset = load_dataset(path_files, labels, 3)

for data in dataset.take(1):
  images, labels = data
  break

print(images.numpy()[0], labels.numpy()[0])
print(images.numpy()[1], labels.numpy()[1])
print(images.numpy()[2], labels.numpy()[2])

# path = "/content/drive/MyDrive/세종교육/CNN/data"
# files = os.listdir(path)
# path_files = [path+f"/{file}" for file in files]
# print(path_files)
# labels = [0 for _ in range(len(path_files))]
# print(labels)
# dataset = load_dataset(path_files, labels, 3)
# for data in dataset.take(1):
#     images, labels = data
#     break
# print(images.numpy()[0], labels.numpy()[0])
# print(images.numpy()[1], labels.numpy()[1])
# print(images.numpy()[2], labels.numpy()[2])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[[0.23844019 0.18862401 0.18959482]
  [0.24553654 0.20481771 0.21128409]
  [0.16856235 0.12949602 0.14817134]
  ...
  [0.5441866  0.53205425 0.45754442]
  [0.7133387  0.6945887  0.620234  ]
  [0.5058249  0.4858494  0.41152343]]

 [[0.4015376  0.3359739  0.40337202]
  [0.3615694  0.28761107 0.36377528]
  [0.28895718 0.2166724  0.28617877]
  ...
  [0.5587067  0.5274146  0.45874694]
  [0.59474    0.5577914  0.49167624]
  [0.5460114  0.4984011  0.435656  ]]

 [[0.4201555  0.41882086 0.5159888 ]
  [0.7493164  0.7405331  0.8364277 ]
  [0.77106696 0.7519991  0.84035695]
  ...
  [0.59354514 0.54489315 0.48214805]
  [0.5955882  0.5446078  0.48186275]
  [0.59798944 0.53524435 0.4764208 ]]

 ...

 [[0.3657188  0.3421894  0.38140512]
  [0.2745768  0.23413564 0.2715935 ]
  [0.7304975  0.6741249  0.7138863 ]
  ...
  [0.19201134 0.15671721 0.23588389]
  [0.18529795 0.15378179 0.2078259 ]
  [0.16205767 0.13545114 0.16102941]]

 [[0.43958718 0.41997933 0.4474303 ]
  [